In [1]:
!pip install selenium
!pip install bs4

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# 우리가 데이터를 가져올 웹 페이지입니다.
faker_opgg_url = 'https://www.op.gg/summoner/userName=Hide%20on%20bush'

In [4]:
# 페이지의 html 코드를 faker_html로 저장합니다.
faker_html = requests.get(faker_opgg_url).text

In [5]:
with open('faker_thml.txt', encoding = 'UTF8') as f:
    text = f.readlines()

In [6]:
faker_html2 = ''.join(text)

In [7]:
faker_soup2 = BeautifulSoup(faker_html2, 'html.parser')

In [8]:
faker_soup2

 <!DOCTYPE html>

<!--
	  ,ad8888ba,   88888888ba         ,ad8888ba,    ,ad8888ba,
	 d8"'    `"8b  88      "8b       d8"'    `"8b  d8"'    `"8b
	d8'        `8b 88      ,8P      d8'           d8'
	88          88 88aaaaaa8P'      88            88
	88          88 88""""""'        88      88888 88      88888
	Y8,        ,8P 88               Y8,        88 Y8,        88
	 Y8a.    .a8P  88                Y8a.    .a88  Y8a.    .a88
	  `"Y8888Y"'   88     dev@op.gg   `"Y88888P"    `"Y88888P"

		 Copyright © 2013-2021 OP.GG. All rights reserved.
-->
<html lang="en_US">
<head>
<script>
		(function(t, l) {
			try {
				let w = window,
						d = document,
						s = d.createElement("script"),
						f = d.getElementsByTagName("script")[0];
				w[t] =
						w[t] ||
						function() {
							(w["_rgea"] = w["_rgea"] || [["uts", new Date()]]).push(
									Array.prototype.slice.call(arguments)
							);
							w.RGEA && w.RGEA.p();
						};
				s.type = "text/javascript";
				s.async = true;
				s.defe

In [9]:
# html 형식에 맞춰 파싱(parsing; 추후 이용하기 쉽도록 쪼개기)합니다.
faker_soup = BeautifulSoup(faker_html, "html.parser")

In [10]:
# 결과가 들어갈 빈 리스트를 만듭니다.
faker_recent_champions = []
faker_recent_kills = []
faker_recent_deaths = []
faker_recent_assists = []
faker_recent_results = []

In [21]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
#select // 리스트 형식.. select_one // 하나만..
faker_recent_games_html = faker_soup2.select('div.GameItemList div.GameItemWrap')
faker_recent_games_html1 = faker_soup2.select_one('div.GameItemList div.GameItemWrap')

In [22]:
#select_one으로 게임 하나만 들고 올 수도 있다!
faker_recent_games_html1

<div class="GameItemWrap">
<div class="GameItem Win" data-game-id="5476558459" data-game-result="win" data-game-time="1632603204" data-summoner-id="4460427">
<div class="Content">
<div class="GameStats">
<div class="GameType" title="Ranked Solo">
					Ranked Solo
				</div>
<div class="TimeStamp"><span class="_timeago _timeCount" data-datetime="1632603204" data-interval="60" data-type="">2021-09-26 05:53:24</span></div>
<div class="Bar"></div>
<div class="GameResult">
											Victory									</div>
<div class="GameLength">23m 58s</div>
</div>
<div class="GameSettingInfo">
<div class="ChampionImage">
<a href="/champion/ekko/statistics" target="_blank"><img alt="Ekko" class="Image" src="//opgg-static.akamaized.net/images/lol/champion/Ekko.png?image=c_scale,q_auto,w_46&amp;v=1634696426"/></a>
</div>
<div class="SummonerSpell">
<div class="Spell">
<img alt="Smite" class="Image tip" src="//opgg-static.akamaized.net/images/lol/spell/SummonerSmite.png?image=c_scale,q_auto,w_22&amp;v=163469

In [12]:
# 그렇게 가져온 html 코드의 element 개수 == 한 페이지에 최초로 로딩된 최근 게임 수 출력
recent_game_len = len(faker_recent_games_html)
print(recent_game_len)

20


In [17]:
list(faker_recent_games_html[0].select('div.ChampionName')[0].stripped_strings)

['Ekko']

In [14]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
for i in range(recent_game_len):
    faker_recent_champions.append(''.join(list(faker_recent_games_html[i].select('div.ChampionName')[0].
                                               stripped_strings)))
    faker_recent_kills.append(list(faker_recent_games_html[i].select('div.KDA div.KDA span.Kill')[0].
                                               stripped_strings)[0])
    faker_recent_deaths.append(list(faker_recent_games_html[i].select_one('div.KDA div.KDA span.Death').
                                               stripped_strings)[0])
    faker_recent_assists.append(list(faker_recent_games_html[i].select('div.KDA div.KDA span.Assist')[0].
                                               stripped_strings)[0])
    faker_recent_results.append(list(faker_recent_games_html[i].select('div.GameStats div.GameResult')[0].
                                               stripped_strings)[0])
#     faker_recent_results.append(faker_recent_games_html[i].select_one('div.GameItem')['data-game-result'])

In [15]:
# DataFrame으로 변환 후 출력
faker_recent_df = pd.DataFrame([faker_recent_champions,
                                faker_recent_results,
                                faker_recent_kills,
                                faker_recent_deaths,
                                faker_recent_assists],
                               index = ['champion', 'result', 'kills', 'deaths', 'assists']).T
faker_recent_df

,champion,result,kills,deaths,assists
0,Ekko,Victory,4,7,11
1,LeBlanc,Defeat,2,2,5
2,LeBlanc,Victory,6,4,4
3,Taliyah,Defeat,5,10,7
4,LeBlanc,Victory,8,5,10
5,Lee Sin,Victory,7,3,11
6,Orianna,Defeat,1,5,9
7,Zoe,Victory,6,0,1
8,Tryndamere,Defeat,7,3,7
9,Lee Sin,Remake,1,0,0
